In [21]:
import os

PATH = os.path.abspath("test.ipynb")
DATA = os.path.join(os.path.dirname(PATH), "data/output_optical_7.txt")

HBANK  = "header bank"
UHBANK = "user header bank"
IRBANK = "integrated raw bank"
IDBANK = "integrated digitized bank"
GPBANK = "generated particles bank"

f = open(DATA)

In [22]:
# Store metadata as strings (for now).
metadata = {}

for i in range(2):
    devnull = f.readline() # Ignore the first two lines.

while True:
    x = f.tell() # Define a stopping point for later (a bit dirty but it's not too bad).
    l = f.readline()
    if l[3] != '>': break # From this point we begin reading events.
    sl = l.split(' ')
    if len(sl) == 7:
        metadata[sl[5]] = sl[6][:-1]
devnull = f.seek(x)

In [53]:
# Store event data in "struct":
#     event_data[0] : header bank (10).
#     event_data[1] : user header bank (currently empty). Assumed to have same format as header bank.
#     event_data[2] : integrated raw bank (51).
#     event_data[3] : integrated digitized bank (52).
#     event_data[4] : generated particles bank.
event_data = {
    HBANK  : {},
    UHBANK : {},
    IRBANK : {},
    IDBANK : {},
    GPBANK : {},
}
bank = None
eof = 0 # End of file checker.

while True:
    l = f.readline()
    l = l[:-1]
    if l == " ---- End of Event  ----": 
        break
    if l == '':
        eof = 1
        break

    # Make sure we're writing to the correct address.
    if   l == " --- Header Bank --":                          bank = HBANK
    elif l == " --- User Header Bank --":                     bank = UHBANK
    elif l == "   -- integrated true infos bank  (51, 0) --": bank = IRBANK
    elif l == "   -- integrated digitized bank  (52, 0) --":  bank = IDBANK
    elif l == " --- Generated Particles Bank --":             bank = GPBANK

    if bank != GPBANK:
        sl = l.split('\t')
        if len(sl) == 1: continue # Ignore lines with titles & additional information.

    if bank == HBANK or bank == UHBANK: # Header & user header banks.
        event_data[bank][sl[0].split(' ')[-1][:-1]] = sl[1]
    if bank == IRBANK or bank == IDBANK: # Raw & Digitized banks.
        event_data[bank][sl[0].split(' ')[-1][:-1]] = sl[1:]
    if bank == GPBANK: # Generated particles bank.
        sl = l.split()
        if sl[1] == 'Particle':
            event_data[bank][sl[ 3][:-1]] = sl[ 4]
            event_data[bank][sl[ 6][:-1]] = sl[ 7]
            event_data[bank][sl[10][:-1]] = sl[11]
        if sl[1] == 'Hit':
            event_data[bank]['nhits'] = sl[ 4]
            event_data[bank][sl[ 7]]  = sl[ 8]
            event_data[bank][sl[11]]  = sl[12]

if eof == 0: print("Event #%s stored!" % (event_data[HBANK]['evn']))
else: print("Reached end of file!")

Reached end of file!


In [26]:
import pprint
pp = pprint.PrettyPrinter(indent=2, compact=True)

pp.pprint(event_data)

{ 'generated particles bank': { 'etot': '7.99223234153',
                                'mom': '(-142.885929537,-9998.97727204,6.09298228938)',
                                'nhits': '138',
                                'pid': '-13',
                                'time': '0.142968276924',
                                'vert': '(0,50,0)'},
  'header bank': { 'beamPol': '0',
                   'evn': '1',
                   'evn_type': '-1',
                   'runNo': '1',
                   'time': 'mié, 05.13.2020 19:39:33'},
  'integrated digitized bank': { 'hitn': [ '1', '2', '3', '4', '5', '6', '7',
                                           '8', '9', '10', '11', '12', '13',
                                           '14', '15', '16', '17', '18', '19',
                                           '20', '21', '22', '23', '24', '25',
                                           '26', '27', '28', '29', '30', '31',
                                           '32', '33', '34', '35',